# Project 4 Hurricane Damage ANN

Ashton Cole

AVC687

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook performs a hyperparameter sweep of a densely-connected Artificial Neural Network applied to a hurricane damage set. Satellite images of properties and houses are classified as either being damaged or undamaged.

*Note: This notebook may take a long time to run. It trains and tests 16 dense neural networks.*

In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Rescaling#

2024-04-25 02:04:16.750603: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 02:04:16.790043: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 02:04:16.790083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 02:04:16.791728: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 02:04:16.799175: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 02:04:16.800045: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_s = 128

train_dir = 'data/split/train'
test_dir = 'data/split/test'

rescale = Rescaling(scale=1.0/255)

data_train, data_validation = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset='both',
    seed=seed,
    image_size=(image_s, image_s),
    batch_size=batch_size)
data_test = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed=seed,
    image_size=(image_s, image_s),
    batch_size=batch_size)

data_train_scaled = data_train.map(lambda image, label: (rescale(image), label))
data_validation_scaled = data_validation.map(lambda image, label: (rescale(image), label))
data_test_scaled = data_test.map(lambda image, label: (rescale(image), label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.
Found 4265 files belonging to 2 classes.


## Model Construction and Compilation

In [3]:
image_length = image_s * image_s * 3

optimizer='adam'
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(image_s, image_s, 3))) # F1
    model.add(Dense(hp.Choice('first', [64, 128, 192, 256]), activation='relu')) # F2
    model.add(Dense(hp.Choice('second', [64, 128, 192, 256]), activation='relu')) # F3
    model.add(Dense(2, activation='softmax')) # Output
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )
    return model

## Model Parameter Search

In [4]:
directory = 'tuner'
project_name = 'ann-hurricane'

tuner = kt.GridSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    overwrite=False,
    directory=directory,
    project_name=project_name
)

tuner.search_space_summary()

Search space summary
Default search space size: 2
first (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}
second (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}


In [5]:
logger = CSVLogger(f'{project_name}.csv', append=True)

tuner.search(
    x=data_train_scaled,
    epochs=10,
    validation_data=data_validation_scaled,
    verbose=2,
    batch_size=batch_size,
    callbacks=[logger]
)

Trial 16 Complete [00h 04m 47s]
val_accuracy: 0.7056581377983093

Best val_accuracy So Far: 0.7836411595344543
Total elapsed time: 00h 48m 19s


## Model Testing

In [6]:
trials = tuner.oracle.trials

for (id, trial) in trials.items():
    print(f'Trial {id}')
    print(trial.summary())
    model = tuner.load_model(trial)
    test_loss, test_accuracy = model.evaluate(data_test_scaled, verbose=0)
    print(f'Test accuracy: {test_accuracy}')
    print('--------------------')

Trial 0000
Trial 0000 summary
Hyperparameters:
first: 64
second: 64
Score: 0.7833479642868042
None
Test accuracy: 0.7681125402450562
--------------------
Trial 0001
Trial 0001 summary
Hyperparameters:
first: 64
second: 128
Score: 0.7836411595344543
None
Test accuracy: 0.7704572081565857
--------------------
Trial 0002
Trial 0002 summary
Hyperparameters:
first: 64
second: 192
Score: 0.7296980619430542
None
Test accuracy: 0.7245017290115356
--------------------
Trial 0003
Trial 0003 summary
Hyperparameters:
first: 64
second: 256
Score: 0.668132483959198
None
Test accuracy: 0.6644783020019531
--------------------
Trial 0004
Trial 0004 summary
Hyperparameters:
first: 128
second: 64
Score: 0.7452359795570374
None
Test accuracy: 0.7399765253067017
--------------------
Trial 0005
Trial 0005 summary
Hyperparameters:
first: 128
second: 128
Score: 0.7789504528045654
None
Test accuracy: 0.7702227234840393
--------------------
Trial 0006
Trial 0006 summary
Hyperparameters:
first: 128
second: 192
S